In [24]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import cv2
import os
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras.models import Sequential
from keras.models import load_model
import h5py as h5

Using TensorFlow backend.


In [2]:
import os
os.getcwd() #to get current working directory

'/Users/ashishsingh/AIML/Assignments/Introduction to Neural Networks and Deep Learning_R7_INN_Project2/Introduction to Neural Networks and Deep Learning_R7_INN_Project2'

Understand the basic Image Classification pipeline and the data-driven approach (train/predict stages)

In [3]:
import h5py
filename = 'SVHN_single_grey1.h5'
dataFrame = h5py.File(filename, 'r')


In [4]:
# # List all groups
print("Keys: %s" % dataFrame.keys())

Keys: <KeysViewHDF5 ['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']>


In [5]:
datasetnames = [n for n in dataFrame.keys()]

In [6]:
for n in datasetnames:
    print(n) 

X_test
X_train
X_val
y_test
y_train
y_val


In [7]:
x_test = dataFrame['X_test']
x_train = dataFrame['X_train']
x_val = dataFrame['X_val']
y_test = dataFrame['y_test']
y_train = dataFrame['y_train']
y_val = dataFrame['y_val']

In [8]:
for i in dataFrame.keys():
    print(dataFrame[i])

<HDF5 dataset "X_test": shape (18000, 32, 32), type "<f4">
<HDF5 dataset "X_train": shape (42000, 32, 32), type "<f4">
<HDF5 dataset "X_val": shape (60000, 32, 32), type "<f4">
<HDF5 dataset "y_test": shape (18000,), type "|u1">
<HDF5 dataset "y_train": shape (42000,), type "|u1">
<HDF5 dataset "y_val": shape (60000,), type "|u1">


In [9]:
dataFrame['X_train']

<HDF5 dataset "X_train": shape (42000, 32, 32), type "<f4">

In [10]:
len(list(dataFrame['X_train']))

42000

In [11]:
len(list(dataFrame['X_train'])[0])

32

In [12]:
list(dataFrame['X_train'])[0]

array([[ 33.0704,  30.2601,  26.852 , ...,  71.4471,  58.2204,  42.9939],
       [ 25.2283,  25.5533,  29.9765, ..., 113.0209, 103.3639,  84.2949],
       [ 26.2775,  22.6137,  40.4763, ..., 113.3028, 121.775 , 115.4228],
       ...,
       [ 28.5502,  36.212 ,  45.0801, ...,  24.1359,  25.0927,  26.0603],
       [ 38.4352,  26.4733,  23.2717, ...,  28.1094,  29.4683,  30.0661],
       [ 50.2984,  26.0773,  24.0389, ...,  49.6682,  50.853 ,  53.0377]],
      dtype=float32)

# Implement and apply an optimal k-Nearest Neighbor (kNN) classifier

In [14]:
len(cv2.resize(list(dataFrame['X_train'])[0], (32,32)).flatten())

1024

In [15]:
# flattening from 3D to 2D
X_train = np.reshape(x_train,(42000,32*32))

In [16]:
NNH = KNeighborsClassifier(n_neighbors=10)

In [17]:
NNH.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [18]:
# predict the model
# Change the dimension from 3D -> 2D
X_test = np.reshape(x_test,(18000,32*32))


In [19]:
prediction = NNH.predict(X_test)

In [20]:
#Accuracy
print("Accuracy:", metrics.accuracy_score(prediction,y_test))

Accuracy: 0.5168333333333334


In [21]:
#Confusion matrix
print("Confusion Matrix:", metrics.confusion_matrix(prediction,y_test))

Confusion Matrix: [[1240   94  110  134  119  160  304  102  257  343]
 [  62 1344  236  265  246  173  118  239  124  144]
 [  47   64  991  145   57   69   44  134   72   73]
 [  38   85  113  719   58  288   75   83  132  107]
 [  47   79   45   46 1154   71  132   32   93   74]
 [  46   39   31  161   26  671  137   29  124   94]
 [ 111   31   30   48   57  135  743   30  254   63]
 [  26   41  133   53   15   23   23 1090   22   65]
 [  91   21   46   90   39  107  201   28  636  126]
 [ 106   30   68   58   41   71   55   41   98  715]]


In [22]:
#Classification report
cr=metrics.classification_report(y_test,prediction)
print(cr)

              precision    recall  f1-score   support

           0       0.43      0.68      0.53      1814
           1       0.46      0.74      0.56      1828
           2       0.58      0.55      0.57      1803
           3       0.42      0.42      0.42      1719
           4       0.65      0.64      0.64      1812
           5       0.49      0.38      0.43      1768
           6       0.49      0.41      0.45      1832
           7       0.73      0.60      0.66      1808
           8       0.46      0.35      0.40      1812
           9       0.56      0.40      0.46      1804

   micro avg       0.52      0.52      0.52     18000
   macro avg       0.53      0.52      0.51     18000
weighted avg       0.53      0.52      0.51     18000



### Implement and apply a deep neural network classifier including (feedforward neural network, RELU activations)

In [25]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

In [26]:
#Reshape data from 2D to 1D -> 32X32 to 1024
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32)))

### Implement batch normalization for training the neural network

In [27]:
#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))

In [29]:
#Output layer
model.add(tf.keras.layers.Dense(10, activation='softmax', name='Output'))

### Understand and be able to implement (vectorized) backpropagation (cost stochastic gradient descent, cross entropy loss, cost functions)

In [30]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics

In [31]:
# Change train and test labels into one-hot vectors
trainY = tf.keras.utils.to_categorical(y_train, num_classes=10)
testY = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [32]:
#Train the model
features_val_arr = np.array(x_test)
model.fit(x_train,trainY,          
          validation_data=(features_val_arr, testY),
          epochs=75,
          batch_size=150,validation_split = 0.01,
         shuffle='batch')

Train on 42000 samples, validate on 18000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/75
42000/42000 [==============================] - 3s 60us/sample - loss: 2.1309 - acc: 0.2624 - val_loss: 1.8501 - val_acc: 0.4179
Epoch 2/75
42000/42000 [==============================] - 2s 59us/sample - loss: 1.6557 - acc: 0.4990 - val_loss: 1.4562 - val_acc: 0.5754
Epoch 3/75
42000/42000 [==============================] - 2s 52us/sample - loss: 1.3465 - acc: 0.6068 - val_loss: 1.2351 - val_acc: 0.6379
Epoch 4/75
42000/42000 [==============================] - 2s 51us/sample - loss: 1.1752 - acc: 0.6564 - val_loss: 1.1119 - val_acc: 0.6785
Epoch 5/75
42000/42000 [==============================] - 2s 52us/sample - loss: 1.0700 - acc: 0.6886 - val_loss: 1.0350 - val_acc: 0.6965
Epoch 6/75
42000/42000 [==============================] - 2s 52us/sample - loss: 0.9965 - acc: 0.7075 - val_loss: 0.9701 - val_acc: 0.7176
Epoch 7/75
42000/42000 [==============================] - 2s 55us/sa

42000/42000 [==============================] - 2s 52us/sample - loss: 0.3270 - acc: 0.9050 - val_loss: 0.5895 - val_acc: 0.8398
Epoch 58/75
42000/42000 [==============================] - 2s 51us/sample - loss: 0.3235 - acc: 0.9055 - val_loss: 0.5850 - val_acc: 0.8438
Epoch 59/75
42000/42000 [==============================] - 2s 51us/sample - loss: 0.3195 - acc: 0.9065 - val_loss: 0.6062 - val_acc: 0.8366
Epoch 60/75
42000/42000 [==============================] - 2s 52us/sample - loss: 0.3174 - acc: 0.9069 - val_loss: 0.6129 - val_acc: 0.8367
Epoch 61/75
42000/42000 [==============================] - 2s 52us/sample - loss: 0.3159 - acc: 0.9070 - val_loss: 0.5924 - val_acc: 0.8420
Epoch 62/75
42000/42000 [==============================] - 2s 51us/sample - loss: 0.3086 - acc: 0.9099 - val_loss: 0.6035 - val_acc: 0.8404
Epoch 63/75
42000/42000 [==============================] - 2s 51us/sample - loss: 0.3078 - acc: 0.9110 - val_loss: 0.5867 - val_acc: 0.8453
Epoch 64/75
42000/42000 [=======

In [34]:
# model evaluation
evaluate = model.evaluate(x_test, testY)
print(evaluate)

18000/18000 [==============================] - 1s 56us/sample - loss: 0.5871 - acc: 0.8493
[0.5871379801829656, 0.84933335]


In [35]:
# predict the model
y_predict=model.predict_classes(x_test)

In [36]:
cr=metrics.classification_report(y_test,y_predict)
print(cr)

              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1814
           1       0.85      0.86      0.85      1828
           2       0.88      0.86      0.87      1803
           3       0.77      0.83      0.80      1719
           4       0.89      0.88      0.89      1812
           5       0.83      0.83      0.83      1768
           6       0.85      0.83      0.84      1832
           7       0.88      0.88      0.88      1808
           8       0.83      0.81      0.82      1812
           9       0.84      0.83      0.83      1804

   micro avg       0.85      0.85      0.85     18000
   macro avg       0.85      0.85      0.85     18000
weighted avg       0.85      0.85      0.85     18000



### Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics

In [16]:
# def image_to_feature_vector(image, size=(32, 32)):
# 	# resize the image to a fixed size, then flatten the image into
# 	# a list of raw pixel intensities
# 	return cv2.resize(image, size).flatten()

In [ ]:
# # train and evaluate a k-NN classifer on the histogram
# # representations
# print("[INFO] evaluating accuracy...")

# NNH.fit(X_train_new, y_train)
# acc = NNH.score(X_train_new, y_train)
# print("[INFO]  accuracy: {:.2f}%".format(acc * 100))

In [ ]:
#classification metrics report
# sklearn.metrics.classification_report(y_test, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False)

In [ ]:
# # K fold
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# # creating odd list of K for KNN
# myList = list(range(1,50))


# # empty list that will hold cv scores
# cv_scores = []
# k_neighbors = []

# # perform 10-fold cross validation
# for k in myList:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     scores = cross_val_score(knn, X_train_new, y_train, cv=10, scoring='accuracy')
#     cv_scores.append(scores.mean())
#     k_neighbors.append(k)


# MSE = [1 - x for x in cv_scores]
# min(MSE)
# MSE.index(min(MSE))
# best_k = myList[MSE.index(min(MSE))]
# print ("The optimal number of neighbors is %d" % best_k)
